In [1]:
import os

from pythae.pipelines import TrainingPipeline
from pythae.trainers import BaseTrainerConfig
from pythae.trainers.training_callbacks import WandbCallback
from pythae.models import AutoModel, BetaVAE, BetaVAEConfig

from cnn_framework.dummy_vae_model.data_set import DummyVAEDataSet
from cnn_framework.dummy_vae_model.model_params import DummyVAEModelParams

from cnn_framework.dummy_vae_model.decoder import CustomDecoder
from cnn_framework.dummy_vae_model.encoder import CustomEncoder

from cnn_framework.utils.DataManagers import DefaultDataManager
from cnn_framework.utils.data_loader_generators.DataLoaderGenerator import DataLoaderGenerator
from cnn_framework.utils.metrics import MeanSquaredErrorMetric
from cnn_framework.utils.model_managers.ModelManager import ModelManager
from cnn_framework.utils.model_managers.VAEModelManager import VAEModelManager
from cnn_framework.utils.create_dummy_data_set import generate_data_set

In [2]:
params = DummyVAEModelParams()
params.update()

# Create data set if needed
if not os.path.exists(params.data_dir):
    generate_data_set(params.data_dir)
    print(f"\nData set created in {params.data_dir}")

print(f"\nModel will be saved in {params.models_folder}")
print(f"Predictions will be saved in {params.output_dir}")
print(f"Tensorboard logs will be saved in {params.tensorboard_folder_path}")

Model time id: 20230901-125835-local
epochs 10 | batch 2 | lr 0.0001 | weight decay 0.05 | dropout 0.0 | latent dim 16 | beta 1 | gamma 0 | delta 0 | depth 5 | kld loss standard | encoder name timm-efficientnet-b0

Model will be saved in C:\Users\thoma\models\local/dummy_vae/20230901-125835-local
Predictions will be saved in C:\Users\thoma\predictions\local/dummy_vae/20230901-125835-local
Tensorboard logs will be saved in C:\Users\thoma\tensorboard\local/20230901-125835-local_dummy_vae


In [3]:
loader_generator = DataLoaderGenerator(params, DummyVAEDataSet, DefaultDataManager)
train_dl, val_dl, test_dl = loader_generator.generate_data_loader()

### Data source ###
train data is loaded from C:\Users\thoma\data\dummy - 80% elements
val data is loaded from C:\Users\thoma\data\dummy - 10% elements
test data is loaded from C:\Users\thoma\data\dummy - 10% elements
###################
train has 160 images.
val has 20 images.
test has 20 images.
###################


In [4]:
# Create folder to save model
os.makedirs(params.models_folder, exist_ok=True)

my_training_config = BaseTrainerConfig(
    output_dir=params.models_folder,
    num_epochs=params.num_epochs,
    learning_rate=params.learning_rate,
    per_device_train_batch_size=params.batch_size,
    per_device_eval_batch_size=params.batch_size,
    train_dataloader_num_workers=params.num_workers,
    eval_dataloader_num_workers=params.num_workers,
    steps_saving=None,
    optimizer_cls="AdamW",
    optimizer_params={
        "weight_decay": params.weight_decay,
        "betas": (params.beta1, params.beta2),
    },
    scheduler_cls="ReduceLROnPlateau",
    scheduler_params={"patience": 5, "factor": 0.5},
)

# Set up the model configuration
my_vae_config = BetaVAEConfig(
    reconstruction_loss=params.reconstruction_loss,
    input_dim=(
        params.nb_modalities * params.nb_stacks_per_modality,
        params.input_dimensions.height,
        params.input_dimensions.width,
    ),
    latent_dim=params.latent_dim,
    beta=params.beta,
    uses_default_decoder=False,
    uses_default_encoder=False,
)

# Build the model
if params.model_pretrained_path:
    vae_model = AutoModel.load_from_folder(params.model_pretrained_path)
    # Update modifiable parameters
    vae_model.model_config = my_vae_config
    vae_model.beta = my_vae_config.beta
else:
    encoder = CustomEncoder(params, my_vae_config)
    print(f"Number of parameters in encoder: {sum(p.numel() for p in encoder.parameters())}")
    decoder = CustomDecoder(params, my_vae_config)
    print(f"Number of parameters in decoder: {sum(p.numel() for p in decoder.parameters())}")
    vae_model = BetaVAE(encoder=encoder, decoder=decoder, model_config=my_vae_config)

# Build the Pipeline
pipeline = TrainingPipeline(training_config=my_training_config, model=vae_model)

# Compute mean_std for future normalization
model_manager = ModelManager(vae_model, params, None)
model_manager.compute_and_save_mean_std(train_dl, val_dl)

train_dl.dataset.initialize_transforms()
val_dl.dataset.initialize_transforms()

# Create you callback
callbacks = []  # the TrainingPipeline expects a list of callbacks
wandb_cb = WandbCallback()  # Build the callback
# SetUp the callback
wandb_cb.setup(
    training_config=my_training_config,  # training config
    model_config=my_vae_config,  # model config
    project_name=params.wandb_project,  # specify your wandb project
    entity_name=params.wandb_entity,  # specify your wandb entity
    run_name=params.format_now,  # name of the run
)
callbacks.append(wandb_cb)  # Add it to the callbacks list

Number of parameters in encoder: 4171420
Number of parameters in decoder: 2498675
Current commit hash: 7095d245630fd90132aa69fe7e290b923dbf4076


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thomas-bonte (cbio-bis). Use `wandb login --relogin` to force relogin


In [5]:
# Launch the Pipeline
pipeline(
    train_data=train_dl.dataset,  # must be torch.Tensor, np.array or torch datasets
    eval_data=val_dl.dataset,  # must be torch.Tensor, np.array or torch datasets
    callbacks=callbacks,
)

Checking train dataset...
Checking eval dataset...
Using Base Trainer

Model passed sanity check !
Ready for training.

Created C:\Users\thoma\models\local/dummy_vae/20230901-125835-local\BetaVAE_training_2023-09-01_12-59-00. 
Training config, checkpoints and final model will be saved here.

Training params:
 - max_epochs: 10
 - per_device_train_batch_size: 2
 - per_device_eval_batch_size: 2
 - checkpoint saving every: None
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.91, 0.995)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.05
)
Scheduler: <torch.optim.lr_scheduler.ReduceLROnPlateau object at 0x000001BD2F4C83A0>

Successfully launched training !



Training of epoch 1/10:   0%|          | 0/80 [00:00<?, ?batch/s]

Eval of epoch 1/10:   0%|          | 0/10 [00:00<?, ?batch/s]

--------------------------------------------------------------------------
Train loss: 44598.9599
Eval loss: 40014.0264
--------------------------------------------------------------------------


Training of epoch 2/10:   0%|          | 0/80 [00:00<?, ?batch/s]

...

--------------------------------------------------------------------------
Train loss: 17182.9861
Eval loss: 10276.1426
--------------------------------------------------------------------------


Training of epoch 10/10:   0%|          | 0/80 [00:00<?, ?batch/s]

Eval of epoch 10/10:   0%|          | 0/10 [00:00<?, ?batch/s]

--------------------------------------------------------------------------
Train loss: 13834.6286
Eval loss: 13711.0205
--------------------------------------------------------------------------
Training ended!
Saved final model in C:\Users\thoma\models\local/dummy_vae/20230901-125835-local\BetaVAE_training_2023-09-01_12-59-00\final_model


eval/epoch_loss,█▆▃▆▄▃▄▃▁▂
train/epoch_loss,█▅▃▂▂▂▁▂▂▁
train/global_step,▁▂▃▃▄▅▆▆▇█
eval/epoch_loss,13711.02046
train/epoch_loss,13834.62864
train/global_step,10


In [6]:
# Test and save images
manager = VAEModelManager(vae_model, params, MeanSquaredErrorMetric)
manager.predict(test_dl)

Current commit hash: 7095d245630fd90132aa69fe7e290b923dbf4076
Model evaluation in progress: 100.0% | Batch #9
Average MeanSquaredError: -0.33
